# 镶嵌多张图像

In [ ]:
import glob
import os
from osgeo import gdal
import math

# 获得x，y的坐标最小值，最大值；即清楚了图像的长和宽。
def get_extent(fn): 
    '''Returns min_x, max_y, max_x, min_y''' 
    ds = gdal.Open(fn)
    gt = ds.GetGeoTransform()
    # ds.RasterXSize 指照片的长有多少像素点，ds.RasterYSize是照片的宽有多少像素点
    return (gt[0], gt[3], gt[0] + gt[1] * ds.RasterXSize, gt[3] + gt[5] * ds.RasterYSize)

os.chdir(r'E:\Tools\QGIS_tutorial_data\osgeopy-data\Massachusetts') 

# Calculate output extent from all inputs
in_files = glob.glob('O*.tif') # 以O开头的所有tif文件
min_x, max_y, max_x, min_y = get_extent(in_files[0]) 

for fn in in_files[1:]: 
    minx, maxy, maxx, miny = get_extent(fn) 
    min_x = min(min_x, minx) 
    max_y = max(max_y, maxy) 
    max_x = max(max_x, maxx) 
    min_y = min(min_y, miny)

# Calculate dimensions
in_ds = gdal.Open(in_files[0]) 
gt = in_ds.GetGeoTransform() 
rows = math.ceil((max_y - min_y) / -gt[5]) 
columns = math.ceil((max_x - min_x) / gt[1])

# Create output
driver = gdal.GetDriverByName('gtiff') 
out_ds = driver.Create('mosaic.tif', columns, rows) 
print("columns = ", columns)
print("rows = ", rows)
out_ds.SetProjection(in_ds.GetProjection()) 
out_band = out_ds.GetRasterBand(1)

# Calculate new geotransform
gt = list(in_ds.GetGeoTransform()) 
gt[0], gt[3] = min_x, max_y 
out_ds.SetGeoTransform(gt)

for fn in in_files: 
    in_ds = gdal.Open(fn) 
    # Get output offsets
    trans = gdal.Transformer(in_ds, out_ds, []) 
    success, xyz = trans.TransformPoint(False, 0, 0) 
    x, y, z = map(int, xyz) 
    # Copy data
    data = in_ds.GetRasterBand(1).ReadAsArray() 
    out_band.WriteArray(data, x, y)

del in_ds, out_band, out_ds

columns =  16345
rows =  14700


# 镶嵌两张图像
拼接O4207063.NES.916802.tif 和 O4207063.NWS.916803.tif

In [6]:
import os
from osgeo import gdal
import math

def get_extent(fn): 
    '''Returns min_x, max_y, max_x, min_y''' 
    ds = gdal.Open(fn)
    gt = ds.GetGeoTransform()
    # ds.RasterXSize 指照片的长有多少像素点，ds.RasterYSize是照片的宽有多少像素点
    return (gt[0], gt[3], gt[0] + gt[1] * ds.RasterXSize, gt[3] + gt[5] * ds.RasterYSize)

os.chdir(r'E:\Tools\QGIS_tutorial_data\osgeopy-data\Massachusetts') 
in_files = ['O4207063.NWS.916803.tif', 'O4207063.NES.916802.tif']
min_x, max_y, max_x, min_y = get_extent(in_files[0]) 

for fn in in_files[1:]: 
    minx, maxy, maxx, miny = get_extent(fn) 
    min_x = min(min_x, minx) 
    max_y = max(max_y, maxy) 
    max_x = max(max_x, maxx) 
    min_y = min(min_y, miny)

# Calculate dimensions
in_ds = gdal.Open(in_files[0]) 
gt = in_ds.GetGeoTransform() 
rows = math.ceil((max_y - min_y) / -gt[5]) 
columns = math.ceil((max_x - min_x) / gt[1])
print(rows, columns)

# Create output
driver = gdal.GetDriverByName('gtiff') 
out_ds = driver.Create('my_first_mosaic.tif', columns, rows) 
out_ds.SetProjection(in_ds.GetProjection()) 
out_band = out_ds.GetRasterBand(1)

# Calculate new geotransform
gt = list(in_ds.GetGeoTransform()) 
gt[0], gt[3] = min_x, max_y 
out_ds.SetGeoTransform(gt)

for fn in in_files: 
    in_ds = gdal.Open(fn) 
    # Get output offsets
    trans = gdal.Transformer(in_ds, out_ds, []) 
    success, xyz = trans.TransformPoint(False, 0, 0) 
    x, y, z = map(int, xyz) 
    # Copy data
    data = in_ds.GetRasterBand(1).ReadAsArray() 
    out_band.WriteArray(data, x, y)

del in_ds, out_band, out_ds

7695 11078


# 随便拿两张图，是否能够镶嵌成功？

In [ ]:
import os
from osgeo import gdal
import math

def get_extent(fn): 
    '''Returns min_x, max_y, max_x, min_y''' 
    ds = gdal.Open(fn)
    gt = ds.GetGeoTransform()
    # ds.RasterXSize 指照片的长有多少像素点，ds.RasterYSize是照片的宽有多少像素点
    return (gt[0], gt[3], gt[0] + gt[1] * ds.RasterXSize, gt[3] + gt[5] * ds.RasterYSize)

os.chdir(r'E:\Tools\QGIS_tutorial_data\osgeopy-data\Massachusetts') 
in_files = ['O4207063.NES.916802.tif', 'random_img_for_test.tif']
min_x, max_y, max_x, min_y = get_extent(in_files[0]) 
for fn in in_files[1:]: 
    minx, maxy, maxx, miny = get_extent(fn) 
    min_x = min(min_x, minx) 
    max_y = max(max_y, maxy) 
    max_x = max(max_x, maxx) 
    min_y = min(min_y, miny)

# Calculate dimensions
in_ds = gdal.Open(in_files[0]) 
gt = in_ds.GetGeoTransform() 
rows = math.ceil((max_y - min_y) / -gt[5]) 
columns = math.ceil((max_x - min_x) / gt[1])
print(rows, columns)

# Create output
driver = gdal.GetDriverByName('gtiff') 

out_ds = driver.Create('unrelated_picture_mosaic.tif', columns, rows) 
print("out_ds is ", out_ds)
out_ds.SetProjection(in_ds.GetProjection()) 
out_band = out_ds.GetRasterBand(1)

# Calculate new geotransform
gt = list(in_ds.GetGeoTransform()) 
gt[0], gt[3] = min_x, max_y 
out_ds.SetGeoTransform(gt)

for fn in in_files: 
    in_ds = gdal.Open(fn) 
    # Get output offsets
    trans = gdal.Transformer(in_ds, out_ds, []) 
    success, xyz = trans.TransformPoint(False, 0, 0) 
    x, y, z = map(int, xyz) 
    # Copy data
    data = in_ds.GetRasterBand(1).ReadAsArray() 
    out_band.WriteArray(data, x, y)

del in_ds, out_band, out_ds

712559 252197
out_ds is  None


AttributeError: 'NoneType' object has no attribute 'SetProjection'